In [1]:
import pandas as pd
import seaborn as sns
import fbprophet 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
# total population by county (adjusted to 2010 controls)
county_2010=pd.read_csv('../../data/NHGIS/nhgis0001_csv/nhgis0001_ts_geog2010_county.csv')
# total population by place
pop_by_place=pd.read_csv('../../data/NHGIS/nhgis0002_csv/nhgis0002_ts_nominal_place.csv',encoding='ISO-8859-1')
# total population by county (1790 to 2010)
by_state_county=pd.read_csv('../../data/NHGIS/nhgis0003_csv/nhgis0003_ts_nominal_county.csv',encoding='ISO-8859-1')

In [4]:
# explore by_state_county 
# expect total population by county 1790-2010
by_state_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4050 entries, 0 to 4049
Data columns (total 30 columns):
GISJOIN      4050 non-null object
STATE        4050 non-null object
STATEFP      3478 non-null float64
STATENH      4050 non-null int64
COUNTY       4050 non-null object
COUNTYFP     3252 non-null float64
COUNTYNH     4050 non-null int64
A00AA1790    289 non-null float64
A00AA1800    424 non-null float64
A00AA1810    582 non-null float64
A00AA1820    761 non-null float64
A00AA1830    994 non-null float64
A00AA1840    1280 non-null float64
A00AA1850    1623 non-null float64
A00AA1860    2077 non-null float64
A00AA1870    2291 non-null float64
A00AA1880    2612 non-null float64
A00AA1890    2802 non-null float64
A00AA1900    2857 non-null float64
A00AA1910    2959 non-null float64
A00AA1920    3074 non-null float64
A00AA1930    3109 non-null float64
A00AA1940    3108 non-null float64
A00AA1950    3111 non-null float64
A00AA1960    3133 non-null float64
A00AA1970    3141 non-null flo

In [5]:
by_state_county.describe()

,STATEFP,STATENH,COUNTYFP,COUNTYNH,A00AA1790,A00AA1800,A00AA1810,A00AA1820,A00AA1830,A00AA1840,...,A00AA1920,A00AA1930,A00AA1940,A00AA1950,A00AA1960,A00AA1970,A00AA1980,A00AA1990,A00AA2000,A00AA2010
count,3478.000000,4050.000000,3252.000000,4050.000000,289.000000,424.000000,582.000000,761.000000,994.000000,1280.000000,...,3.074000e+03,3.109000e+03,3.108000e+03,3.111000e+03,3.133000e+03,3.141000e+03,3.137000e+03,3.141000e+03,3.141000e+03,3.221000e+03
mean,31.350201,305.952099,103.736777,1006.067654,17959.930796,13853.483491,13643.687285,12643.385020,12938.120724,13330.744531,...,3.448977e+04,3.962775e+04,4.252417e+04,4.864217e+04,5.723689e+04,6.469657e+04,7.221734e+04,7.918175e+04,8.959628e+04,9.701066e+04
std,16.472735,164.901946,107.726294,1258.369511,49775.843869,23831.416467,24433.485140,13364.409992,15107.002777,17627.906255,...,1.078691e+05,1.350028e+05,1.436520e+05,1.689053e+05,2.035123e+05,2.282614e+05,2.361912e+05,2.638127e+05,2.924622e+05,3.092988e+05
min,1.000000,10.000000,1.000000,5.000000,305.000000,233.000000,29.000000,147.000000,5.000000,18.000000,...,3.700000e+01,5.200000e+01,0.000000e+00,0.000000e+00,4.700000e+01,6.400000e+01,9.100000e+01,5.200000e+01,6.700000e+01,8.200000e+01
25%,19.000000,180.000000,35.000000,270.000000,6503.000000,5372.500000,4756.250000,4263.000000,4133.500000,4598.000000,...,1.028075e+04,1.017800e+04,1.046125e+04,1.000850e+04,9.233000e+03,9.085000e+03,1.031300e+04,1.033200e+04,1.120600e+04,1.131000e+04
50%,30.000000,290.000000,79.000000,690.000000,10203.000000,9389.000000,9037.000000,9029.000000,8560.500000,8690.000000,...,1.776000e+04,1.766200e+04,1.864500e+04,1.867400e+04,1.818000e+04,1.845500e+04,2.160300e+04,2.208500e+04,2.459500e+04,2.607600e+04
75%,47.000000,460.000000,133.000000,1270.000000,17572.000000,16094.000000,15057.000000,15820.000000,16288.500000,15804.250000,...,2.872475e+04,3.053000e+04,3.301650e+04,3.570800e+04,3.908500e+04,4.233100e+04,5.063500e+04,5.475300e+04,6.175800e+04,6.588000e+04
max,72.000000,720.000000,840.000000,9997.000000,747610.000000,422845.000000,472040.000000,137097.000000,202589.000000,312710.000000,...,3.053017e+06,3.982123e+06,4.063342e+06,4.508792e+06,6.038771e+06,7.032075e+06,7.477503e+06,8.863164e+06,9.519338e+06,9.818605e+06


- ***notes***:
    - looks to be in line with other data
- ***the txt***:

    Context Fields 
        GISJOIN:     GIS Join Match Code
        STATE:       NHGIS Integrated State Name
        STATEFP:     FIPS State Code
        STATENH:     NHGIS Integrated State Code
        COUNTY:      NHGIS Integrated County Name
        COUNTYFP:    FIPS County Code
        COUNTYNH:    NHGIS Integrated County Code
 
    Table 1: (A00) Total Population
        Time series AA: Persons: Total
            A00AA1790:   1790: Persons: Total
            A00AA1800:   1800: Persons: Total
            A00AA1810:   1810: Persons: Total
            A00AA1820:   1820: Persons: Total
            A00AA1830:   1830: Persons: Total
            A00AA1840:   1840: Persons: Total
            A00AA1850:   1850: Persons: Total
            A00AA1860:   1860: Persons: Total
            A00AA1870:   1870: Persons: Total
            A00AA1880:   1880: Persons: Total
            A00AA1890:   1890: Persons: Total
            A00AA1900:   1900: Persons: Total
            A00AA1910:   1910: Persons: Total
            A00AA1920:   1920: Persons: Total
            A00AA1930:   1930: Persons: Total
            A00AA1940:   1940: Persons: Total
            A00AA1950:   1950: Persons: Total
            A00AA1960:   1960: Persons: Total
            A00AA1970:   1970: Persons: Total
            A00AA1980:   1980: Persons: Total
            A00AA1990:   1990: Persons: Total
            A00AA2000:   2000: Persons: Total
            A00AA2010:   2010: Persons: Total

In [17]:
by_state_county.head(3)

,GISJOIN,STATE,STATEFP,STATENH,COUNTY,COUNTYFP,COUNTYNH,A00AA1790,A00AA1800,A00AA1810,...,A00AA1920,A00AA1930,A00AA1940,A00AA1950,A00AA1960,A00AA1970,A00AA1980,A00AA1990,A00AA2000,A00AA2010
0,G0100010,Alabama,1.0,10,Autauga County,1.0,10,NaN,NaN,NaN,...,18908.0,19694.0,20977.0,18186.0,18739.0,24460.0,32259.0,34222.0,43671.0,54571.0
1,G0100015,Alabama,1.0,10,Baker County,NaN,15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G0100030,Alabama,1.0,10,Baldwin County,3.0,30,NaN,NaN,NaN,...,20730.0,28289.0,32324.0,40997.0,49088.0,59382.0,78556.0,98280.0,140415.0,182265.0


In [14]:
# GISJOIN:     GIS Join Match Code
print(len(by_state_county.GISJOIN.unique()),len(by_state_county.GISJOIN))

#   STATE:       NHGIS Integrated State Name
print(len(by_state_county.STATE.unique()),len(by_state_county.STATE))

#   STATEFP:     FIPS State Code
print(len(by_state_county.STATEFP.unique()),len(by_state_county.STATEFP))

#   STATENH:     NHGIS Integrated State Code
print(len(by_state_county.STATENH.unique()),len(by_state_county.STATENH))

#   COUNTY:      NHGIS Integrated County Name
print(len(by_state_county.COUNTY.unique()),len(by_state_county.COUNTY))

#   COUNTYFP:    FIPS County Code
print(len(by_state_county.COUNTYFP.unique()),len(by_state_county.COUNTYFP))

#   COUNTYNH:    NHGIS Integrated County Code
print(len(by_state_county.COUNTYNH.unique()),len(by_state_county.COUNTYNH))

4050 4050
81 4050
53 4050
81 4050
2186 4050
342 4050
551 4050


- ***action***:
    - keep GISJOIN, drop else

In [71]:
# GIS Join Match Code
a=by_state_county.copy()[['GISJOIN','A00AA1790','A00AA1800','A00AA1810','A00AA1820','A00AA1830','A00AA1840','A00AA1850','A00AA1860',
               'A00AA1870','A00AA1880','A00AA1890','A00AA1900','A00AA1910','A00AA1920','A00AA1930','A00AA1940','A00AA1950',
                'A00AA1960','A00AA1970','A00AA1980','A00AA1990','A00AA2000','A00AA2010']]
len(a.dropn(axis=1,thresh=10))

AttributeError: 'DataFrame' object has no attribute 'dropnan'

In [64]:
state_county_ = by_state_county.copy()[['COUNTY','STATE']]
state_county_names = [state_county_.loc[i][0] + ', ' + state_county_.loc[i][1] for i in range(len(state_county_))]


['Autauga County, Alabama',
 'Baker County, Alabama',
 'Baldwin County, Alabama',
 'Barbour County, Alabama',
 'Benton County, Alabama',
 'Bibb County, Alabama',
 'Blount County, Alabama',
 'Bullock County, Alabama',
 'Butler County, Alabama',
 'Cahawba County, Alabama',
 'Calhoun County, Alabama',
 'Chambers County, Alabama',
 'Cherokee County, Alabama',
 'Chilton County, Alabama',
 'Choctaw County, Alabama',
 'Clarke County, Alabama',
 'Clay County, Alabama',
 'Cleburne County, Alabama',
 'Coffee County, Alabama',
 'Colbert County, Alabama',
 'Conecuh County, Alabama',
 'Coosa County, Alabama',
 'Cotaco County, Alabama',
 'Covington County, Alabama',
 'Crenshaw County, Alabama',
 'Cullman County, Alabama',
 'Dale County, Alabama',
 'Dallas County, Alabama',
 'DeKalb County, Alabama',
 'Elmore County, Alabama',
 'Escambia County, Alabama',
 'Etowah County, Alabama',
 'Fayette County, Alabama',
 'Franklin County, Alabama',
 'Geneva County, Alabama',
 'Greene County, Alabama',
 'Hale Co

In [ ]:
nyc=pop_by_place.iloc[[18672]]